In [12]:
import cv2
import pdf2image
from PIL import Image
import numpy as np
from pytesseract import pytesseract
import matplotlib as mpl
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
import re

mpl.rcParams['image.cmap'] = 'gray'

path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
pytesseract.tesseract_cmd = path_to_tesseract

In [13]:
start_pdf = 2
end_pdf = 2

In [14]:
def convert_pdf_to_image(file, dpi):
    images_list = []
    images = pdf2image.convert_from_path(file, dpi = dpi, poppler_path = r"C:\Users\Amandeep Singh\Downloads\Release-23.05.0-0\poppler-23.05.0\Library\bin")
    
    for i, image in enumerate(images):
        image_np = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        images_list.append(image_np)
        
    return images_list

In [15]:
def f(x):
    global horizontal_lines_img, verticle_lines_img, kernel
    kernel_length = np.array(img).shape[1]//x

    verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))
    hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    img_temp1 = cv2.erode(img_bin, verticle_kernel, iterations=3)
    verticle_lines_img = cv2.dilate(img_temp1, verticle_kernel, iterations=3)
    
    cv2.imwrite("Images/verticle_lines.jpg",verticle_lines_img)
    img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
    horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)

In [16]:
def sort_contours(cnts, method):
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),key=lambda b:[b[1][1],b[1][0]] , reverse=False))
    return (cnts, boundingBoxes)

In [17]:
print("started")
for pdf_file in range(start_pdf, end_pdf+1):
    images = convert_pdf_to_image(str(pdf_file)+".pdf", 300)
    
    final_list = []
    count = 0
    for i in range(2, len(images)-1):
        img = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)
        
        (thresh, img_bin) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        img_bin = 255 - img_bin
        cv2.imwrite("Images/Image_bin.jpg", img_bin)

        interact(f, x = widgets.IntSlider(min = 1, max = 255, step = 1, value = 50));
        
        alpha = 0.5
        beta = 1.0 - alpha
        img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)
        img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)
        (thresh, img_final_bin) = cv2.threshold(img_final_bin, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        
        contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        (contours, boundingBoxes) = sort_contours(contours,"top-to-bottom")
        
        g = []
        for c in contours:
            x, y, w, h = cv2.boundingRect(c)
            if(w < 800 and w > 500 and h > 250):
                g.append([x, y, w, h])
                
        a = []
        for i in range(len(g)):
            if i == 0:
                a.append(g[i])
            else:
                if np.abs(g[i][0] - g[i-1][0]) < 50:
                    pass
                else:
                    a.append(g[i])
                
        for i in a:
            new_img = img[i[1]:i[1]+i[3], i[0]:i[0]+i[2]]
            dic = {}
                        
            extracted_text = pytesseract.image_to_string(new_img, lang='kan', config='--oem 3 --psm 6')
            extracted_text = extracted_text.split()
            
            #Getting name from the list
            start_index_for_name = None
            end_index_for_name = None
            for i, item in enumerate(extracted_text):
                if (item.startswith("ಹೆಸರ") or item.startswith("ಹಸರ")) and start_index_for_name is None:
                    start_index_for_name = i
                elif (item.startswith("ತಂದೆ") or item.startswith("ಗಂಡ") or item.startswith('ಮನೆ') or item.startswith("ತೆಂದೆ") or item.startswith("ತಾಯಿ")) and end_index_for_name is None:
                    end_index_for_name = i
                    break
                    
            if start_index_for_name is not None and end_index_for_name is not None:
                name = ' '.join(extracted_text[start_index_for_name + 1:end_index_for_name])
                pattern_for_name_filter = str.maketrans('', '', '೦೧೨೩೪೫೬0123456789/:ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz')
                name = name.translate(pattern_for_name_filter)
                
                patterns = r"\u200c"
                name = re.sub(patterns, "", name)
                
                name = name.strip()
                dic["Name"] = name
            else:
                name = "None"
                
            #Getting relation_name from the list
            start_index_for_relation = None
            end_index_for_relation = None
            relation = None
            for i, item in enumerate(extracted_text):
                if (item.startswith('ತಂದೆ') or item.startswith('ತೆಂದೆ')) and start_index_for_relation is None:
                    relation = "Father"
                    start_index_for_relation = i
                    break
                elif item.startswith('ಗಂಡ') and start_index_for_relation is None:
                    relation = "Husband"
                    start_index_for_relation = i
                    break
                elif item.startswith('ತಾಯಿ') and start_index_for_relation is None:
                    relation = "Mother"
                    start_index_for_relation = i
                    break
                    
            for i, item in enumerate(extracted_text):
                if (item.startswith('ಮನೆ') or item.startswith('ಮನೆ')) and end_index_for_relation is None:
                    end_index_for_relation = i
                    break
    
            if start_index_for_relation is not None and start_index_for_relation is not None:
                relation_name =  ' '.join(extracted_text[start_index_for_relation + 1:end_index_for_relation])
                pattern_for_relation_filter = str.maketrans('', '', '೦೧೨೩೪೫೬0123456789/:ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz')
                relation_name = relation_name.translate(pattern_for_relation_filter)
                
                patterns = r"ಹೆಸರು: |\u200c|ಹೆಸರು"
                relation_name = re.sub(patterns, "", relation_name)
                relation_name = relation_name.strip()
                
            if relation == "Father":
                dic["FatherName"] = relation_name
            else:
                dic["FatherName"] = ""
                
            if relation == "Husband":
                dic["HusbandName"] = relation_name
            else:
                dic["HusbandName"] = ""
                
            if relation == "Mother":
                dic["MotherName"] = relation_name
            else:
                dic["MotherName"] = ""
            
            dic["num"] = count
            print(dic)
            final_list.append(dic)
            count += 1
            
    sd = pd.DataFrame(final_list)
    sd[["Name", "FatherName", "HusbandName", "MotherName"]].to_excel(str(pdf_file) + ".xlsx")
    
    print(f"{pdf_file} completed")
    
print("===")
print("end")

started


interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

{'Name': 'ಥಮಾಬಾಯಿ ತಿರೊಳೆ', 'FatherName': '', 'HusbandName': 'ಹರಿ', 'MotherName': '', 'num': 0}
{'Name': 'ರಾದಾಬಾಯಿ ಕಾಂಬಳೆ', 'FatherName': '', 'HusbandName': 'ಶ್ರೀಪತಿ ಬ', 'MotherName': '', 'num': 1}
{'Name': 'ಬಾಳಾಸೊಲೋಹಾರ', 'FatherName': 'ರಾಯಗಾಡ ತ್', 'HusbandName': '', 'MotherName': '', 'num': 2}
{'Name': 'ವಿಲಾಸ ಕಾಂಬಳೆ', 'FatherName': 'ಜಾನಬಾ  ಓ', 'HusbandName': '', 'MotherName': '', 'num': 3}
{'Name': 'ಮಹಾದೇವ ಸಾಲ್ಕೆ', 'FatherName': 'ದತ್ತು', 'HusbandName': '', 'MotherName': '', 'num': 4}
{'Name': 'ಗಣವಂತಿ ಕಂಬಾರ', 'FatherName': '', 'HusbandName': 'ನಾಮದೇವ', 'MotherName': '', 'num': 5}
{'Name': 'ಲಕ್ಷ್ಮಿ ಲೋಹಾರ', 'FatherName': '', 'HusbandName': 'ಬಾಳಾಸೊ', 'MotherName': '', 'num': 6}
{'Name': 'ಬಾಳಾಸೋ ಮಗದುಮ್ಮ', 'FatherName': 'ರಾಮಚಂದ್ರ  ಇ', 'HusbandName': '', 'MotherName': '', 'num': 7}
{'Name': 'ಪಾಂಡುವನಿರೆ', 'FatherName': 'ಬಾಳು', 'HusbandName': '', 'MotherName': '', 'num': 8}
{'Name': 'ಮಂದಾಕಿನಿ ವನಿರೆ', 'FatherName': '', 'HusbandName': 'ಪಾಂಡು ೯ ಇ', 'MotherName': '', 'num': 9}
{'Name': 'ಉತ್ತಮ ಪವಾರ',

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

{'Name': 'ಸವಿತಾನಲವಡೆ', 'FatherName': '', 'HusbandName': 'ಅನೀಲ', 'MotherName': '', 'num': 30}
{'Name': 'ರಾಜಶ್ರೀ ಕಾಂಬಳೆ', 'FatherName': '', 'HusbandName': 'ರವೀಂದ್ರ', 'MotherName': '', 'num': 31}
{'Name': 'ನೀಲಕಂಠ ಗೊರಡೆ', 'FatherName': 'ಯಶವಂತ', 'HusbandName': '', 'MotherName': '', 'num': 32}
{'Name': 'ಸುನಂದಾ ಪಾಟೀಲ', 'FatherName': 'ಚೇತನ', 'HusbandName': '', 'MotherName': '', 'num': 33}
{'Name': 'ಬಾರತಿ ಮಗೆದುವ', 'FatherName': '', 'HusbandName': 'ಅಣ್ಣಾಸೋ', 'MotherName': '', 'num': 34}
{'Name': 'ಪ್ರವೀಣ ಪಾಟೀಲ', 'FatherName': 'ಆತ್ಮಾರಾಮ', 'HusbandName': '', 'MotherName': '', 'num': 35}
{'Name': 'ವಿಶಾಲ ನಲವಡೆ', 'FatherName': 'ಅಶೋಕ', 'HusbandName': '', 'MotherName': '', 'num': 36}
{'Name': 'ದಿಗಂಬರ ಕಾಂಬಳೆ', 'FatherName': 'ಪ್ರಲ್ಹಾದ', 'HusbandName': '', 'MotherName': '', 'num': 37}
{'Name': 'ಶುಭಾಂಗಿ ಪಾಟೀಲ', 'FatherName': 'ರಾಜಾರಾಮ', 'HusbandName': '', 'MotherName': '', 'num': 38}
{'Name': 'ಅನೀಲ ಪಾಟೀಲ', 'FatherName': 'ಶಿವಾಜಿ  ಓ', 'HusbandName': '', 'MotherName': '', 'num': 39}
{'Name': 'ಅಜೀತ ಕಾಂಬಳೆ', 'Fat

...

{'Name': 'ಪದ್ಮವತಿ ಪಾಟೀಲ', 'FatherName': '', 'HusbandName': 'ರಾಮದಾಸ ಟ್', 'MotherName': '', 'num': 1067}
{'Name': 'ರಾಜಶ್ರೀ ಪೋವಾರ', 'FatherName': '', 'HusbandName': 'ಅಪ್ಪಾಸೊ', 'MotherName': '', 'num': 1068}
{'Name': 'ಛಾಯಾ ಪೋವಾರ', 'FatherName': '', 'HusbandName': 'ಮಧುಕರ', 'MotherName': '', 'num': 1069}
{'Name': 'ಕೃಷ್ಣಾತ ಪೊವಾರ', 'FatherName': 'ಬಳವಂತ', 'HusbandName': '', 'MotherName': '', 'num': 1070}
{'Name': 'ಸುವರ್ಣಾ ಮಗದುಮ್', 'FatherName': '', 'HusbandName': 'ಮಲಗೊಂಡಾ', 'MotherName': '', 'num': 1071}
{'Name': 'ಶಾಲನ ನಲ್ಲವಡೆ', 'FatherName': '', 'HusbandName': 'ಗಣಪತಿ', 'MotherName': '', 'num': 1072}
{'Name': 'ರುಶಿಕೇಷ ಮಗದುಮ್ಮ', 'FatherName': 'ಅಪ್ಪಾಸಾಹೇಬ', 'HusbandName': '', 'MotherName': '', 'num': 1073}
{'Name': 'ಚಂದ್ರಕಾಂತ ಶಿರೊಳೆ', 'FatherName': 'ಬಾಬುರಾವ', 'HusbandName': '', 'MotherName': '', 'num': 1074}
{'Name': 'ಪಾರುಬಾಯಿ ಲೋಹಾರ', 'FatherName': '', 'HusbandName': 'ಮಾರುತಿ', 'MotherName': '', 'num': 1075}
{'Name': 'ಕಾಜಲ ಪಾಟೀಲ', 'FatherName': 'ವಿಕಾಸ ಟ್', 'HusbandName': '', 'MotherName': '', 'num

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

{'Name': 'ಸಿದ್ದಾರ್ಥ ಕಾಂಬಳೆ', 'FatherName': 'ಗಣಪತಿ ಕಾಂಬಳೆ', 'HusbandName': '', 'MotherName': '', 'num': 1089}
{'Name': 'ಅನುಸೂಯ ಕಾಂಬಳೆ', 'FatherName': '', 'HusbandName': 'ಶಿವಾಜಿ ಕಾಂಬಳೆ', 'MotherName': '', 'num': 1090}
{'Name': 'ಓಂಕಾರ ಮಗದುಮ್ಮ', 'FatherName': 'ಅರವಿಂದ   ಮಗದುಮ್ಮ ಓಜ', 'HusbandName': '', 'MotherName': '', 'num': 1091}
{'Name': 'ವಿಘ್ನೇಶ್ ತಿರುವತ', 'FatherName': 'ತಾನಾಜಿ ತಿರುವತ', 'HusbandName': '', 'MotherName': '', 'num': 1092}
{'Name': 'ಅನುರಾಗ ಕಾಂಬಳೆ', 'FatherName': 'ರಮೇಶ ಕಾಂಬಳೆ ಫ್', 'HusbandName': '', 'MotherName': '', 'num': 1093}
{'Name': 'ವಿವೇಕ ಪಾಟೀಲ', 'FatherName': 'ಅಶೋಕ ಪಾಟೀಲ ಅ ಲ್', 'HusbandName': '', 'MotherName': '', 'num': 1094}
{'Name': 'ಶಂತನುಶಿರೋಳೆ', 'FatherName': 'ಸುಹಾಸ ಶಿರೋಳೆ ೭', 'HusbandName': '', 'MotherName': '', 'num': 1095}
{'Name': 'ಶಂಭುರಾಜ ಮಾನೆ', 'FatherName': 'ಶಶಿಕಾಂತ ಮಾನೆ', 'HusbandName': '', 'MotherName': '', 'num': 1096}
{'Name': 'ಅಮ್ರಪಾಲಿ ಕಾಂಬಳೆ', 'FatherName': 'ಅಶೋಕ ಕಾಂಬಳೆ', 'HusbandName': '', 'MotherName': '', 'num': 1097}
{'Name': 'ಆರತಿ ಮಗದುಮ್ಮ', 'Fat

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

{'Name': 'ವೈಭವ್', 'FatherName': 'ಹರಿಶ ಪೋವರ್', 'HusbandName': '', 'MotherName': '', 'num': 1110}
{'Name': 'ವೈಭವ್', 'FatherName': 'ರವಸಾಹೇಬ   ಮಗ್ದೂಮ ಓ', 'HusbandName': '', 'MotherName': '', 'num': 1111}
{'Name': '', 'FatherName': '', 'HusbandName': 'ಮಹದೇವ್ ಕಾಮ್ಬಲೇ', 'MotherName': '', 'num': 1112}
{'Name': 'ಜ್ಯೊತಿ', 'FatherName': '', 'HusbandName': 'ಅಮೋಲ ೭ ಮೇರ್ದಾಣೇ ಹಯಂ', 'MotherName': '', 'num': 1113}
{'Name': 'ಸರಿಕ', 'FatherName': '', 'HusbandName': 'ಸಂತೊಶ್ ಪೋವರ್  ಇ', 'MotherName': '', 'num': 1114}
{'Name': 'ಮಾನಸೀ', 'FatherName': '', 'HusbandName': 'ಅಜಯ ಶಿರೋಲೇ', 'MotherName': '', 'num': 1115}
{'Name': 'ಪಾವನ್', 'FatherName': '', 'HusbandName': '', 'MotherName': 'ಅಂಜನಾ ಪೋವರ್ | ಕ್ಲ', 'num': 1116}
{'Name': 'ಸಮೃದ್ದಿ', 'FatherName': 'ಜ್ಞಾನ್ವೇವ ಪೋವರ್', 'HusbandName': '', 'MotherName': '', 'num': 1117}
{'Name': 'ಸಚಿನ್', 'FatherName': 'ತಾನಾಜೀ ಚವನ್', 'HusbandName': '', 'MotherName': '', 'num': 1118}
{'Name': 'ಮಹೆಶ್', 'FatherName': '', 'HusbandName': '', 'MotherName': 'ಉಜ್ವಾಲ ಕಾಮ್ಬಲ್ಲೆ |', 'num': 11